# Препроцессинг

In [1]:
import openpyxl
import re
import pymystem3
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import gensim
import logging
import nltk.data
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import word2vec
import warnings
warnings.filterwarnings('ignore')

## Тут обработка моих примеров

In [3]:
## Работа с таблицей по векам

wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21'] # Имя лист

In [4]:
m = Mystem()

In [119]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=1001, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' '))
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = re.sub('\[.+\]', '', item)
        item = re.sub(r'\b[Тт]о есть', r'тоесть', item) # \b убирает случаи что есть
        item = re.sub(r'\b[Аа] именно', r'аименно', item)
        item = re.sub(r'\b[Тт]ак как', r'таккак', item)
        item = re.sub(r'\b[Пп]отому что', r'потомучто', item)
        v2.append(item)
values = [v1[i]+v2[i] for i in range(1000)]
print(values[:10])

[' Я засунула пластинку и блюдо в один пакет, положила под сиденье и ― надо знать наши самолёты советских времён, лишённые кондиционеров, ― в конце пути в Ереван достала из-под сиденья пластинку, принявшую форму таза, тоесть блюда.  ', ' Само название места происходит от того, что здесь врачевали «косимых» ― тоесть увечных.  ', ' Я даже не могу найти концов, тоесть понять, кто же мне, собственно, должен платить.  ', 'Никаких провокаций ― известные имена-брэнды.  Расчёт на то, что, даже если зритель не шибкоприобщён к балету и не знает, кто такая Аньес Летестю, он непременно отреагирует на Opera National de Paris.  Или хотя бы на «Лебединое озеро». Или хотя бы на Paris.  Плюс гарантия качества: звёзды подобраны заслуженные, но при этом, что важно, действующие: спешно доставленные напароходе прямо из Парижа, поднимешь крышку ― пар! тоесть готовые доказывать свой статус каждым спектаклем.  ', '  «Чтобы не быть выдавленными с рынка, ― говорит академик Алексей Макаров, ― необходимы жесткий 

In [126]:
# Парсим предложения
def parser(string):
    punctuation = (list(""",;:”$%^&*№()_—=+|[]{}\"/<>`~±§«»°1234567890""")
               + ['- ', ' -', ' - ', '― ', ' ―', ' ― ', " '", "' "])
    string = string.lower()
    for ch in list('.…!?'): # Разбиваем примеры на предложения
        string = string.replace(ch, '\n')
    for char in punctuation:
        string = string.replace(char, ' ')
    lemmas = m.lemmatize(string)
    parsed = ''.join(lemmas)
    return parsed

In [127]:
data = [parser(example) for example in values]

In [128]:
# Записываем в файл лемматизированные предложения
with open('21_century.txt', 'w', encoding='utf-8') as fw:
    for i in data:
        fw.write(i)

# Парсер корпуса

### Нужно:
1. Соединить слова *то есть, а именно, потому что, так как*
2. Убрать омонимию *если, то; не, а*

In [19]:
import pickle
from pprint import pprint

In [20]:
with open('/Volumes/Анна/Corpora/stable/disamb/corpora/lines_1m_to_3m.pickle', 'rb') as f:
    data_13 = pickle.load(f)

In [29]:
with open('/Volumes/Анна/Corpora/stable/disamb/corpora/lines_3000000.pickle', 'rb') as f:
    data_3 = pickle.load(f)

In [30]:
BigData = data_13 + data_3

In [73]:
# Видим точку не в т.к., т.д. или т.е. - новое предложение
text = []  # Список предложений
sentence = []  # Обновляемый список слов в предложении
k = -1  # Счетчик позиции. Вот если у нас "т" из т.е., мы можем не обрабатывать следующие 3 символа
for i in range(len(BigData)-2):
    if i <= k:  # Если счетчик больше, то мы уже учли это слово
        continue
    else:
        k = -1
    if not (BigData[i][0] == '!' or BigData[i][0] == '?' or BigData[i][0] == '.' or BigData[i][0] == '…') :
        if (BigData[i][0] == 'то' or BigData[i][0] == 'То') and BigData[i+1][0] == 'есть':
            sentence.append('тоесть')
            k = i + 1  # Не смотрим на следующее "есть"
        elif (BigData[i][0] == 'так' or BigData[i][0] == 'Так') and BigData[i+1][0] == 'как':
            sentence.append('таккак')
            k = i + 1
        elif (BigData[i][0] == 'потому' or BigData[i][0] == 'Потому') and BigData[i+1][0] == 'что':
            sentence.append('потомучто')
            k = i + 1
        elif (BigData[i][0] == 'А' or BigData[i][0] == 'А') and BigData[i+1][0] == 'именно':
            sentence.append('аименно')
            k = i + 1
        elif (BigData[i][0] == 'т' or BigData[i][0] == 'Т') and BigData[i+1][0] == '.' and BigData[i+2][0] == 'е':
            sentence.append('тоесть')
            k = i + 3 
        elif (BigData[i][0] == 'т' or BigData[i][0] == 'Т') and BigData[i+1][0] == '.' and BigData[i+2][0] == 'к':
            sentence.append('таккак')
            k = i + 3
        else:
            sentence.append(BigData[i][2])
    elif not (BigData[i+1][0] == 'п' or BigData[i+1][0] == 'д'):  # Если у нас точка, и она не входит в "т.п." и "д.р."
        text.append(sentence)
        sentence = []

In [74]:
with open('lems.txt', 'w') as fw:
    for i in text:
        fw.write(' '.join(i).lower())
        fw.write('/n')

# Word2Vec CBOW

In [144]:
f = 'lems.txt'
data = gensim.models.word2vec.LineSentence(f)

In [145]:
model_sbow = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50)

In [146]:
model_sbow.init_sims(replace=True)
model_path = "to.bin"

print("Saving model...")
model_sbow.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [147]:
model_sbow.wv.most_similar_cosmul("тоесть", topn=50)

[('и', 0.6347854733467102),
 ('в', 0.6344821453094482),
 ('таккак', 0.6324251890182495),
 ('быть', 0.6278139352798462),
 ('который', 0.6256133317947388),
 ('но', 0.6248376369476318),
 ('а', 0.6205902695655823),
 ('весь', 0.6196562051773071),
 ('хотя', 0.6196523904800415),
 ('если', 0.618808388710022),
 ('как', 0.6176967024803162),
 ('для', 0.6171594262123108),
 ('например', 0.6157851815223694),
 ('значит', 0.6156123876571655),
 ('либо', 0.6148117184638977),
 ('же', 0.6146326661109924),
 ('установить', 0.614324152469635),
 ('высокий', 0.6116353273391724),
 ('иметь', 0.6106708645820618),
 ('при', 0.6098961234092712),
 (',', 0.6087378263473511),
 ('или', 0.6086727976799011),
 ('причем', 0.60835200548172),
 ('этот', 0.6072604060173035),
 ('потомучто', 0.6063065528869629),
 ('где', 0.6062798500061035),
 ('это', 0.6058995723724365),
 ('–', 0.6058865189552307),
 (')', 0.6044421792030334),
 ('на', 0.604044497013092),
 ('без', 0.6039661169052124),
 ('не', 0.6037483215332031),
 ('кстати', 0.6035

In [112]:
l = 'я здесь самый умный тоесть как  самый умный  недоумевать'

In [113]:
l = l.split()

In [114]:
model_sbow.predict_output_word(l) # Это я не очень поняла как работает, но еще особенно не разбиралась

[('лучший', 3.6221896e-05),
 ('главное', 3.6005662e-05),
 ('дело', 3.525131e-05),
 ('важный', 3.524584e-05),
 ('умный', 3.4789413e-05),
 ('красивый', 3.4572855e-05),
 ('интересный', 3.426596e-05),
 ('слово', 3.350955e-05),
 ('смелый', 3.3074713e-05),
 ('правильный', 3.3052063e-05)]

## Проблемы:
1. Не учитываем омонимию (*Если модель заработает, **то есть** шанс, что все будет хорошо.* и т.п.)
2. Разбиение на предложения портят штуки типа г.Москва, 2000гг. и др.
3. Нужно ли оставлять знаки препинания? Например, тире это что-то типа равнозначности
4. Нужно ли разделять случаи *То есть* и *то есть*?
5. Что-то явно не то с парсингом моих примеров, но я пока не вижу что
6. А сколько вообще ближайших соседей адекватное число?
7. Косинусная близость соседей примерно 0.3, это очень мало, что делать? Тут я поменяла метод измерения, опять же не разобравшись в нем (Простите, не успела пока)
8. А еще не учтены случаи знаков препинания типа *??!!!!*
9. Все-таки пока не видно дискурсивного *то есть*

## Дальше я пробовала работать с другими моделями

Тут все плоховато

# CBOW с моими примерами

Ничего не работает из-за плохо обработанных моих примеров. Я еще поработаю над ними.

In [129]:
with open("corp_table.txt", "w") as fw:
    for i in text:
        fw.write(' '.join(i).lower())
        fw.write('/n')
    for i in data:
        fw.write(i)

In [130]:
f = 'corp_table.txt'
data = gensim.models.word2vec.LineSentence(f)

In [131]:
model_cbow_full = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50, sg=1)

In [132]:
model_cbow_full.init_sims(replace=True)
model_path = "to.bin"

print("Saving model...")
model_cbow_full.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [133]:
model_cbow_full.wv.most_similar("тоесть", topn=50)

[('ароматно', 0.33030757308006287),
 ('загрязнение', 0.3236773610115051),
 ('общенародный', 0.31093573570251465),
 ('суфражистка', 0.30130282044410706),
 ('израсходовать', 0.29774612188339233),
 ('олить', 0.2936965227127075),
 ('цивилизованно', 0.2917793393135071),
 ('11-13', 0.29002130031585693),
 ('поглазеть', 0.28996801376342773),
 ('обратный', 0.28883910179138184),
 ('начитать', 0.2886679768562317),
 ('дж/nверди', 0.28675782680511475),
 ('доллар/nсейчас', 0.28536850214004517),
 ('прогрессивнее', 0.28291743993759155),
 ('мотивировка', 0.28272777795791626),
 ('пропорциональный', 0.2820762097835541),
 ('сужение', 0.279745876789093),
 ('возбудиться', 0.2797427177429199),
 ('dds', 0.2786122262477875),
 ('всесоюзный', 0.2781897783279419),
 (')/nсоответственно', 0.27817827463150024),
 ('передел', 0.2773246765136719),
 ('таргум', 0.2769983410835266),
 ('детерминизм', 0.27667146921157837),
 ('заметнее', 0.2763804793357849),
 ('4-ого', 0.27579981088638306),
 ('таня/n-', 0.27409252524375916),

# Word2Vec Skip-gram

Ничего не работает, потому что это skip-gram

In [115]:
f = 'lems.txt'
data = gensim.models.word2vec.LineSentence(f)

In [116]:
model_to = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50, sg=1)

In [117]:
model_to.init_sims(replace=True)
model_path = "to.bin"

print("Saving model...")
model_to.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [118]:
model_to.wv.most_similar("тоесть", topn=50)

[('ароматно', 0.34522080421447754),
 ('"/nдолгин', 0.3146291971206665),
 ('суфражистка', 0.30915361642837524),
 ('выключатель', 0.30832329392433167),
 ('зальчик', 0.30427107214927673),
 ('комплексный', 0.3041404187679291),
 ('израсходовать', 0.3035739064216614),
 ('ошибка/nв', 0.3016357421875),
 ('350', 0.2975699305534363),
 (')/nсоответственно', 0.2966846823692322),
 ('таня/n-', 0.2957918047904968),
 ('таргум', 0.29400986433029175),
 ('цивилизованно', 0.2903851866722107),
 ('олить', 0.29026418924331665),
 ('запущенность', 0.2897460460662842),
 ('ребяческий', 0.28849926590919495),
 ('отвественность', 0.2881307005882263),
 ('передел', 0.28570765256881714),
 ('загрязнение', 0.2841710150241852),
 ('отождествляться', 0.283867746591568),
 ('inclusive', 0.28291207551956177),
 ('изворачиваться', 0.28290051221847534),
 ('кутч', 0.282699853181839),
 ('существительное', 0.2816200852394104),
 ('переложение', 0.27822941541671753),
 ('деньги/nэто', 0.2775688171386719),
 ('отваливать', 0.27752882242

# GloVe

Тут все совсем странно

In [134]:
with open('lems.txt') as fh:
    lines = fh.readlines()
lines = [line.split() for line in lines]

In [135]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lines, window=10)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=5, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [136]:
print(glove.most_similar('тоесть'))

[(')/nа', 0.9989536978920346), ('вчера', 0.9987213646983383), ('тема', 0.9986575014778142), ('year', 0.9981276952849215)]


# AdaGram

Во втором значении модель говорит, что *то есть* похоже на какие-то редкие полнозначные слова. В первом значении немного получше.

In [140]:
! adagram-train lems.txt out1.pkl

[INFO] 2020-01-25 13:22:59,600 Building dictionary...
[INFO] 2020-01-25 13:28:27,833 Done! 21286 words.
[INFO] 2020-01-25 13:34:48,041 1.67% -6.6546 0.0246 1.2/3.0 0.17 kwords/sec
[INFO] 2020-01-25 13:34:51,640 3.33% -6.4787 0.0242 1.5/4.0 17.76 kwords/sec
[INFO] 2020-01-25 13:34:55,177 5.00% -6.3445 0.0238 1.7/5.0 18.10 kwords/sec
[INFO] 2020-01-25 13:34:58,666 6.67% -6.2380 0.0233 1.7/5.0 18.34 kwords/sec
[INFO] 2020-01-25 13:35:02,195 8.33% -6.1489 0.0229 1.8/5.0 18.14 kwords/sec
[INFO] 2020-01-25 13:35:06,370 10.00% -6.0709 0.0225 1.8/5.0 15.33 kwords/sec
[INFO] 2020-01-25 13:35:09,682 11.66% -6.0008 0.0221 1.8/5.0 19.32 kwords/sec
[INFO] 2020-01-25 13:35:13,123 13.33% -5.9372 0.0217 1.9/5.0 18.60 kwords/sec
[INFO] 2020-01-25 13:35:16,715 15.00% -5.8787 0.0213 1.9/5.0 17.82 kwords/sec
[INFO] 2020-01-25 13:35:20,265 16.66% -5.8252 0.0208 1.9/5.0 18.03 kwords/sec
[INFO] 2020-01-25 13:35:23,521 18.33% -5.7758 0.0204 2.0/5.0 19.65 kwords/sec
[INFO] 2020-01-25 13:35:26,827 20.00% -5.730

In [141]:
import adagram
vm = adagram.VectorModel.load('out1.pkl')

In [142]:
vm.word_sense_probs('тоесть')

[(0, 0.9999310392386732)]

In [148]:
vm.sense_neighbors('тоесть', 0)

[(':', 4, 0.4486275),
 ('мыслить', 0, 0.4284525),
 ('к', 2, 0.4283172),
 ('человеческий', 0, 0.42445052),
 ('тренд', 0, 0.42221954),
 ('некорректно', 0, 0.41890633),
 ('-', 3, 0.4168862),
 ('уничтожать', 0, 0.4129591),
 ('у', 0, 0.40553477),
 ('в', 1, 0.39676043)]